In [1]:
import numpy as np 
import pandas as pd
import os
import math
from tqdm.notebook import tqdm
import glob
import matplotlib.pyplot as plt

In [2]:
def extract_psd(bids_root, keywords, nCh=60):
    files = glob.glob(os.path.join(bids_root,'sub*','eeg','*'+keywords+"*"))
    psd = list()
    for file in tqdm(files):
        data = np.load(file, allow_pickle=True)
        psd.append(data)
        
    return np.array(psd)

def extract_psd_seoul(bids_root, keywords, session,  nCh=60):
    files = glob.glob(os.path.join(bids_root,'sub*',session, 'eeg','*'+keywords+"*"))
    psd = list()
    for file in tqdm(files):
        data = np.load(file, allow_pickle=True)
        psd.append(data)
        
    return np.array(psd)

def extract_matrix_new(bids_root, keywords = 'connectivity'):
    files = glob.glob(os.path.join(bids_root,'sub*','eeg','*'+keywords+"*"))
    wpli = list()
    iplv = list()
    oCC = list()
    cplv = list()
    name_subj=list()
    for file in tqdm(files):
        data = np.load(file, allow_pickle=True)
        name_subj.append(file.split("/")[7])
        
        data = np.abs(data).mean(axis=1)
        _, nCh, _, nF = data.shape 
        
        for d in data:
            for fIdx in range(nF):
                np.fill_diagonal(d[...,fIdx], 0)
                
        _, _, wpli_obs, wpli_sur, oCC_obs, oCC_sur = data
        
        obs_wpli = np.zeros((nF,))
        sur_wpli = np.zeros((nF,))
        obs_occ = np.zeros((nF,))
        sur_occ = np.zeros((nF,))
        for iF in range(nF):
            tmp_wpli_obs = wpli_obs[...,iF]
            tmp_wpli_sur = wpli_sur[...,iF]
            obs_wpli[iF] = np.nanmean(tmp_wpli_obs[tmp_wpli_obs != 0])
            sur_wpli[iF] = np.nanmean(tmp_wpli_sur[tmp_wpli_sur != 0])
            tmp_occ_obs = oCC_obs[...,iF]
            tmp_occ_sur = oCC_sur[...,iF]
            obs_occ[iF] = np.mean(tmp_occ_obs[tmp_wpli_obs != 0])
            sur_occ[iF] = np.mean(tmp_occ_sur[tmp_wpli_sur != 0])
        
        wpli.append([obs_wpli,sur_wpli])     
        oCC.append([obs_occ,sur_occ]) 
            
    wpli = np.array(wpli)
    oCC = np.array(oCC)
    return name_subj, wpli, oCC

def extract_matrix_seoul(bids_root, keywords = 'connectivity'):
    files = glob.glob(os.path.join(bids_root,'sub*','ses-*','eeg','*'+keywords+"*"))
    wpli = list()
    iplv = list()
    oCC = list()
    cplv = list()
    name_subj=list()
    for file in tqdm(files):
        data = np.load(file, allow_pickle=True)
        name_subj.append(file.split("/")[7])
        _, _, _, nF = data.shape 
        for d in data:
            for fIdx in range(nF):
                np.fill_diagonal(d[...,fIdx], 0)
                
        _, _, wpli_obs, wpli_sur, oCC_obs, oCC_sur = data
                
        obs_wpli = np.zeros((nF,))
        sur_wpli = np.zeros((nF,))
        obs_occ = np.zeros((nF,))
        sur_occ = np.zeros((nF,))
        for iF in range(nF):
            tmp_wpli_obs = wpli_obs[...,iF]
            tmp_wpli_sur = wpli_sur[...,iF]
            obs_wpli[iF] = np.nanmean(np.abs(tmp_wpli_obs[tmp_wpli_obs != 0]))
            sur_wpli[iF] = np.nanmean(np.abs(tmp_wpli_sur[tmp_wpli_sur != 0]))
            tmp_occ_obs = oCC_obs[...,iF]
            tmp_occ_sur = oCC_sur[...,iF]
            obs_occ[iF] = np.mean(np.abs(tmp_occ_obs[tmp_wpli_obs != 0]))
            sur_occ[iF] = np.mean(np.abs(tmp_occ_sur[tmp_wpli_sur != 0]))
        
        wpli.append([obs_wpli,sur_wpli])     
        oCC.append([obs_occ,sur_occ])

    wpli = np.array(wpli)
    oCC = np.array(oCC)
    return name_subj, wpli, oCC

def create_frequency_axis(f_min=2, f_max= 4, scale_freq ='log', m=1.1):
    
    if scale_freq == 'log':
        f_min_log = math.floor(np.log10(f_min))
        f_max_log = math.ceil(np.log10(f_max))
        mb = np.logspace(f_min_log, f_max_log, num=40) # Morlet bank 
        freq_vals = mb[(2.1 < mb) & (mb < 90)] # Frequency of interest
        
    
    elif scale_freq == 'lin':
        freq_vals = [f_min]
        while freq_vals[~0] < f_max:
            freq_vals.append(freq_vals[~0]*m)

    freq_labels = [('%.2f'%x) for x in freq_vals]
    
    return freq_vals, freq_labels


def extract_index_and_name(lista, subj, remove=False):
    b, index = list(), list()
    for i, name in enumerate(lista): 
        if remove == False:
            if name in subj: 
                b.append(name)
                index.append(i)
        elif remove == True:
            if name not in subj:
                b.append(name)
                index.append(i)
           
    return index, b  

In [3]:
rbd_root = '/mnt/nas_biolab/data/monica/RBD/RBD_SanMartino/'
ctrl_root = '/mnt/nas_biolab/data/monica/RBD/Controls/Controls_SanMartino/'
rbd_seoul_root = '/mnt/nas_biolab/data/monica/RBD/RBD_Seoul/'
ctrl_seoul_root = '/mnt/nas_biolab/data/monica/RBD/Controls/Controls_Seoul'
path_out = '/home/monica/Documents/'

freq_vals, freq_labels = create_frequency_axis(f_min=2, f_max=90, scale_freq ='log')
freq_vals = freq_vals[:30]


freq_pow = np.linspace(0,39,40)

In [4]:
# Seoul
name_subj_r1_seoul, wplir1_seoul, occr1_seoul = extract_matrix_seoul(bids_root=rbd_seoul_root, keywords = 'ses-01_eeg_connectivity.npy')
psdr1_seoul = extract_psd_seoul(bids_root=rbd_seoul_root, keywords ='power2.npy', session='ses-01')

idx_subj_r1_seoul, name_subj_r1_seoul = extract_index_and_name(name_subj_r1_seoul, ['sub-18', 'sub-35', 'sub-39'], remove=True)
wplir1_seoul = wplir1_seoul[idx_subj_r1_seoul,:,:]
occr1_seoul = occr1_seoul[idx_subj_r1_seoul,:,:]
psdr1_seoul = psdr1_seoul[idx_subj_r1_seoul,:]

name_subj_c1_seoul, wplic1_seoul, occc1_seoul = extract_matrix_seoul(bids_root=ctrl_seoul_root, keywords = 'ses-01_eeg_connectivity.npy')
psdc1_seoul = extract_psd_seoul(bids_root=ctrl_seoul_root, keywords ='power2.npy', session='ses-01')

  0%|          | 0/36 [00:00<?, ?it/s]

<ipython-input-2-aaa901bf4704>:84: RuntimeWarning: Mean of empty slice
  obs_wpli[iF] = np.nanmean(np.abs(tmp_wpli_obs[tmp_wpli_obs != 0]))
<ipython-input-2-aaa901bf4704>:85: RuntimeWarning: Mean of empty slice
  sur_wpli[iF] = np.nanmean(np.abs(tmp_wpli_sur[tmp_wpli_sur != 0]))
/home/monica/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/monica/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [5]:
#Italy
name_subj_r1_italy, wplir1_italy, occr1_italy = extract_matrix_new(bids_root=rbd_root, keywords = 'run-01_connectivity2.npy')
psdr1_italy = extract_psd(bids_root=rbd_root, keywords = 'run-01_power2.npy')

name_subj_c1_italy, wplic1_italy, occc1_italy = extract_matrix_new(bids_root=ctrl_root, keywords ='run-01_connectivity2.npy')
psdc1_italy = extract_psd(bids_root=ctrl_root, keywords = 'run-01_power2.npy')

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [6]:
wpli = np.concatenate((wplir1_seoul, wplir1_italy, wplic1_seoul, wplic1_italy))

occ = np.concatenate((occr1_seoul, occr1_italy, occc1_seoul, occc1_italy))

psd = np.concatenate((psdr1_seoul[:,:,:40].mean(axis=1), psdr1_italy[:,:,:40].mean(axis=1), psdc1_seoul[:,:,:40].mean(axis=1), psdc1_italy[:,:,:40].mean(axis=1)))

In [7]:
group_list_irbd = np.ones((len(wplir1_italy)+len(wplir1_seoul)), dtype='int64')
group_list_ctrl = np.zeros((len(wplic1_italy)+len(wplic1_seoul)), dtype='int64')

group_list = np.concatenate((group_list_irbd, group_list_ctrl)).tolist()

In [8]:
# Only PSD
df_psd = pd.DataFrame()
df_psd['Groups'] = group_list

for i, freq in enumerate(freq_pow):
    name_col = 'psd_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_psd[name_col] = psd[:,i]

df_psd.to_pickle('OnlyPSD.npy')

In [9]:
# Only wPLI
df_wpli = pd.DataFrame()
df_wpli['Groups'] = group_list

for i, freq in enumerate(freq_vals):
    name_col = 'wpli_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_wpli[name_col] = wpli[:,0,i]

df_wpli.to_pickle('OnlyWPLI.npy') 

In [10]:
# Only OCC
df_occ = pd.DataFrame()
df_occ['Groups'] = group_list

for i, freq in enumerate(freq_vals):
    name_col = 'occ_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_occ[name_col] = occ[:,0,i]

df_occ.to_pickle('OnlyOCC.npy') 

In [11]:
# PSD + wPLI
df_psdwpli = pd.DataFrame()
df_psdwpli['Groups'] = group_list

for i, freq in enumerate(freq_pow):
    name_col = 'psd_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_psdwpli[name_col] = psd[:,i]

for i, freq in enumerate(freq_vals):
    name_col = 'wpli_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_psdwpli[name_col] = wpli[:,0,i]

df_psdwpli.to_pickle('PSDWPLI.npy') 

In [12]:
# PSD + OCC
df_psdocc = pd.DataFrame()
df_psdocc['Groups'] = group_list

for i, freq in enumerate(freq_pow):
    name_col = 'psd_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_psdocc[name_col] = psd[:,i]

for i, freq in enumerate(freq_vals):
    name_col = 'occ_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_psdocc[name_col] = occ[:,0,i]

df_psdocc.to_pickle('PSDOCC.npy')

In [13]:
# WPLI + OCC
df_wpliocc = pd.DataFrame()
df_wpliocc['Groups'] = group_list

for i, freq in enumerate(freq_vals):
    name_col = 'wpli_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_wpliocc[name_col] = wpli[:,0,i]

for i, freq in enumerate(freq_vals):
    name_col = 'occ_' + str(np.round(freq, decimals=1)) + 'Hz'
    df_wpliocc[name_col] = occ[:,0,i]

df_wpliocc.to_pickle('WPLIOCC.npy') 

In [14]:
# All
df = pd.DataFrame()
df['Groups'] = group_list

for i, freq in enumerate(freq_pow):
    name_col = 'psd_' + str(np.round(freq, decimals=1)) + 'Hz'
    df[name_col] = psd[:,i]

for i, freq in enumerate(freq_vals):
    name_col = 'wpli_' + str(np.round(freq, decimals=1)) + 'Hz'
    df[name_col] = wpli[:,0,i]

for i, freq in enumerate(freq_vals):
    name_col = 'occ_' + str(np.round(freq, decimals=1)) + 'Hz'
    df[name_col] = occ[:,0,i]

df.to_pickle('AllFeatures.npy') 

<ipython-input-14-481da13cfece>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name_col] = occ[:,0,i]
